In [1]:
import lightgbm as lgb
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import gc
import time

In [2]:
from scipy.sparse import csr_matrix
def save_csr(matrix,file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    np.save(data_file,matrix.data)
    print('data save')
    np.save(indices_file,matrix.indices)
    print('indices save')
    np.save(indptr_file,matrix.indptr)
    print('indptr save')
    print('save done!')
    
def load_csr(file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    data = np.load(data_file)
    print('data load')
    indices = np.load(indices_file)
    print('indices load')
    indptr = np.load(indptr_file)
    print('indptr load')
    csr = csr_matrix((data, indices, indptr))
    return csr

In [5]:
train_csr = load_csr('../../data/features/xgb/train_csr_2500')
predict_csr = load_csr('../../data/features/xgb/predict_csr_2500')
train_y = np.load('../../data/features/xgb/train_y_f620_b70_col17k.npy')
predicted_file = pd.read_csv('../../data/features/xgb/predict_f620_b70_col17k.csv')
instance_train = pd.read_pickle('../../data/features/xgb/train_noclean_bytime.pkl')[['instance_id']].copy()
instance_test = predicted_file[['instance_id']].copy()

data load
indices load
indptr load
data load
indices load
indptr load


# load LibFM

In [16]:
fm_train = pd.read_pickle(FILE.X_fe_train_libfm.value)
fm_test = pd.read_pickle(FILE.X_fe_test_libfm.value)
fm_train = instance_train.merge(fm_train,how='inner',on='instance_id')
fm_train.drop('instance_id',axis=1,inplace=True)
print(fm_train.shape)
fm_test = instance_test.merge(fm_test,how='inner',on='instance_id')
fm_test.drop('instance_id',axis=1,inplace=True)
print(fm_test.shape)

(2992639, 27)
(80276, 28)


In [ ]:
X_fe[col] = pd.cut(X_fe[col], bins=cut_bin, labels=False, retbins=True, right=False)[0]  

In [17]:
train_csr_f = sparse.hstack(
    (sparse.csr_matrix(fm_train), train_csr), 'csr').astype(
    'float32')
predict_csr_f = sparse.hstack(
    (sparse.csr_matrix(fm_test), predict_csr), 'csr').astype('float32')
print(train_csr_f.shape)

(2992639, 2414)


In [36]:
predict_csr_f.shape

(80276, 17027)

# Define lgb model

In [18]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=32, reg_alpha=0, reg_lambda=0.1,
    max_depth=-1, n_estimators=5000, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, random_state=1001, n_jobs=16
)


In [35]:
n_fold = 5
gc.collect()
skf = StratifiedKFold(n_splits=n_fold, random_state=19, shuffle=True)
best_score = []
oof = np.zeros(len(train_y))
for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y)):
#     lgb_model.fit(train_csr_f[train_index], train_y[train_index],
#                   eval_set=[(train_csr_f[train_index], train_y[train_index]),
#                             (train_csr_f[test_index], train_y[test_index])], eval_metric =['logloss'],
#                   early_stopping_rounds=250,verbose=10)
    lgb_model.fit(train_csr_f[train_index], train_y[train_index],
                      eval_set=[(train_csr_f[train_index], train_y[train_index]),
                                (train_csr_f[test_index], train_y[test_index])], early_stopping_rounds=200,verbose=200)
    best_score.append(lgb_model.evals_result_['valid_1']['binary_logloss'][lgb_model.best_iteration_-1])
    print(best_score)
    print('cv:',np.mean(best_score))
    test_pred = lgb_model.predict_proba(predict_csr_f, ntree_limit=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
    oof[test_index] = lgb_model.predict_proba(train_csr_f[test_index], ntree_limit=lgb_model.best_iteration_)[:, 1]
    gc.collect()

predict_result['predicted_score'] = predict_result['predicted_score']/5

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.413915	valid_1's binary_logloss: 0.414385
[400]	valid_0's binary_logloss: 0.411951	valid_1's binary_logloss: 0.414099
[600]	valid_0's binary_logloss: 0.410261	valid_1's binary_logloss: 0.413982
[800]	valid_0's binary_logloss: 0.408649	valid_1's binary_logloss: 0.413889
[1000]	valid_0's binary_logloss: 0.407171	valid_1's binary_logloss: 0.413856
[1200]	valid_0's binary_logloss: 0.405695	valid_1's binary_logloss: 0.4138
[1400]	valid_0's binary_logloss: 0.404302	valid_1's binary_logloss: 0.413785
[1600]	valid_0's binary_logloss: 0.402932	valid_1's binary_logloss: 0.413784
Early stopping, best iteration is:
[1572]	valid_0's binary_logloss: 0.403121	valid_1's binary_logloss: 0.413772
[0.41377217910033426]
cv: 0.41377217910033426


ValueError: Number of features of the model must match the input. Model n_features_ is 2414 and input n_features is 17027 

In [34]:
lgb_model.best_iteration_

1572

In [31]:
lgb_model.evals_result_['valid_1']['binary_logloss'][1572]

0.41377293716000957

In [28]:
def test_on_sample(train_csr_f,train_y_f,lgb_model,n_fold = 3,seed=None,first=1500):
    if seed is None:
        seed = np.random.randint(1000)
    cv_list = []
    skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
    best_score = []


    feature_importance_tmp_split = []
    feature_importance_tmp_gain = []
    df_fea = pd.DataFrame()

    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index], train_y_f[train_index]),
                                (train_csr_f[test_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
        best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_score)
        print('cv:',np.mean(best_score))
        feature_importance_tmp_split.append(lgb_model.booster_.feature_importance(importance_type='split'))
        feature_importance_tmp_gain.append(lgb_model.booster_.feature_importance(importance_type='gain'))

    df_fea['col_name'] = lgb_model.booster_.feature_name()
    df_fea['split'] = np.mean(feature_importance_tmp_split,axis=0)
    df_fea['gain'] = np.mean(feature_importance_tmp_gain,axis=0)

    cv_list.append(np.mean(best_score))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))
    print('after dropping...')


    split_index = list(df_fea[df_fea.split!=0].index)
    gain_index = list(df_fea[df_fea.gain!=0].index)
    #union
#     retain_index = list(set(split_index+gain_index))
    #intersection
#     retain_index = list(set(split_index).intersection(set(gain_index)))
    if first is None:
        first = 500
    split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
    gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
    retain_index = list(set(split_index).intersection(set(gain_index)))
    print(len(retain_index))

    best_scor2 = []
    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index][:,retain_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index][:,retain_index], train_y_f[train_index]),
                                (train_csr_f[test_index][:,retain_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
        best_scor2.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_scor2)
        print('cv:',np.mean(best_scor2))
    cv_list.append(np.mean(best_scor2))
    drop = False
    if cv_list[-1] < cv_list[-2]:
        print('dropping successfully!')
        print('cv drop:{}'.format(cv_list[-2]-cv_list[-1]))
        drop = True
    else:
        print('dropping failed!!!!!!')
        print('cv increase:{}'.format(cv_list[-1]-cv_list[-2]))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))
    return df_fea, retain_index, drop

In [36]:
def verify_on_sample(train_csr_f,train_y_f,lgb_model,retain_index,n_fold = 3,seed=None,cv_list=[]):
    if seed is None:
        seed = np.random.randint(1000)

    skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
    best_score = []
    
    if len(cv_list) == 0:

        t1 = time.time()
        for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
            lgb_model.fit(train_csr_f[train_index], train_y_f[train_index],
                          eval_set=[(train_csr_f[train_index], train_y_f[train_index]),
                                    (train_csr_f[test_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
            best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
            print(best_score)
            print('cv:',np.mean(best_score))




        cv_list.append(np.mean(best_score))
        t2 = time.time()
        print('time spend: {} s'.format(t2-t1))
        print('after dropping...')

        print(len(retain_index))

    best_scor2 = []
    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index][:,retain_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index][:,retain_index], train_y_f[train_index]),
                                (train_csr_f[test_index][:,retain_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
        best_scor2.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_scor2)
        print('cv:',np.mean(best_scor2))
    cv_list.append(np.mean(best_scor2))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))

    if cv_list[-1] < cv_list[-2]:
        print('dropping successfully!')
        print('cv drop:{}'.format(cv_list[-2]-cv_list[-1]))
        return True
    else:
        print('dropping failed!!!!!!')
        print('cv increase:{}'.format(cv_list[-1]-cv_list[-2]))
        return False
    
    

In [27]:
df_fea, retain_index, ifdrop = test_on_sample(samples[0],samples_y[0],lgb_model,n_fold = 5,seed=19)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.41039	valid_1's binary_logloss: 0.418157
[400]	valid_0's binary_logloss: 0.401302	valid_1's binary_logloss: 0.417809
Early stopping, best iteration is:
[398]	valid_0's binary_logloss: 0.401385	valid_1's binary_logloss: 0.417795
[0.41779450975427096]
cv: 0.41779450975427096
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.410001	valid_1's binary_logloss: 0.41907
[400]	valid_0's binary_logloss: 0.400928	valid_1's binary_logloss: 0.418624
Early stopping, best iteration is:
[474]	valid_0's binary_logloss: 0.397968	valid_1's binary_logloss: 0.418523
[0.41779450975427096, 0.4185230659386098]
cv: 0.41815878784644034
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.410198	valid_1's binary_logloss: 0.418205
Early stopping, best iteration is:
[286]	valid_0's binary_logloss: 0.40603	valid_1's binary_logloss

In [25]:
if ifdrop:
    verify_drop = verify_on_sample(samples[1],samples_y[1],lgb_model,retain_index,n_fold = 5,seed=21)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.41176	valid_1's binary_logloss: 0.416903
[400]	valid_0's binary_logloss: 0.402715	valid_1's binary_logloss: 0.416445
[600]	valid_0's binary_logloss: 0.394742	valid_1's binary_logloss: 0.416382
Early stopping, best iteration is:
[507]	valid_0's binary_logloss: 0.398304	valid_1's binary_logloss: 0.416348
[0.41634779832398616]
cv: 0.41634779832398616
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.410971	valid_1's binary_logloss: 0.419463
Early stopping, best iteration is:
[244]	valid_0's binary_logloss: 0.40881	valid_1's binary_logloss: 0.419309
[0.41634779832398616, 0.4193088064896909]
cv: 0.41782830240683855
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.411046	valid_1's binary_logloss: 0.419822
Early stopping, best iteration is:
[284]	valid_0's binary_logloss: 0.406948	valid_1's binary_loglos

In [29]:
df_fea, retain_index, ifdrop = test_on_sample(train_csr,train_y,lgb_model,n_fold = 5,seed=19,first=1500)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.416214	valid_1's binary_logloss: 0.416022
[400]	valid_0's binary_logloss: 0.413937	valid_1's binary_logloss: 0.414851
[600]	valid_0's binary_logloss: 0.412246	valid_1's binary_logloss: 0.414457
[800]	valid_0's binary_logloss: 0.410771	valid_1's binary_logloss: 0.414263
[1000]	valid_0's binary_logloss: 0.409375	valid_1's binary_logloss: 0.414097
[1200]	valid_0's binary_logloss: 0.408044	valid_1's binary_logloss: 0.413993
[1400]	valid_0's binary_logloss: 0.406768	valid_1's binary_logloss: 0.413942
[1600]	valid_0's binary_logloss: 0.405533	valid_1's binary_logloss: 0.413916
Early stopping, best iteration is:
[1518]	valid_0's binary_logloss: 0.406031	valid_1's binary_logloss: 0.413911
[0.41391132240832157]
cv: 0.41391132240832157
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.415903	valid_1's binary_logloss: 0.417085
[400]	valid_0's binary_loglo

[0.4138514986861429, 0.4150293287481048, 0.41502096663759397, 0.4147727589831188]
cv: 0.4146686382637401
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.416008	valid_1's binary_logloss: 0.416479
[400]	valid_0's binary_logloss: 0.413759	valid_1's binary_logloss: 0.415374
[600]	valid_0's binary_logloss: 0.412093	valid_1's binary_logloss: 0.415002
[800]	valid_0's binary_logloss: 0.410603	valid_1's binary_logloss: 0.414837
[1000]	valid_0's binary_logloss: 0.409206	valid_1's binary_logloss: 0.414704
[1200]	valid_0's binary_logloss: 0.407865	valid_1's binary_logloss: 0.414604
[1400]	valid_0's binary_logloss: 0.406566	valid_1's binary_logloss: 0.414562
[1600]	valid_0's binary_logloss: 0.405276	valid_1's binary_logloss: 0.414522
Early stopping, best iteration is:
[1509]	valid_0's binary_logloss: 0.405856	valid_1's binary_logloss: 0.41452
[0.4138514986861429, 0.4150293287481048, 0.41502096663759397, 0.4147727589831188, 0.41451976759681625]
cv: 0.

In [37]:
first=2500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

2387
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.4162	valid_1's binary_logloss: 0.415942
[400]	valid_0's binary_logloss: 0.413925	valid_1's binary_logloss: 0.414764
[600]	valid_0's binary_logloss: 0.412247	valid_1's binary_logloss: 0.414325
[800]	valid_0's binary_logloss: 0.410777	valid_1's binary_logloss: 0.414124
[1000]	valid_0's binary_logloss: 0.409362	valid_1's binary_logloss: 0.413955
[1200]	valid_0's binary_logloss: 0.408001	valid_1's binary_logloss: 0.413864
[1400]	valid_0's binary_logloss: 0.406724	valid_1's binary_logloss: 0.413796
[1600]	valid_0's binary_logloss: 0.405456	valid_1's binary_logloss: 0.413737
[1800]	valid_0's binary_logloss: 0.40423	valid_1's binary_logloss: 0.413708
[2000]	valid_0's binary_logloss: 0.403014	valid_1's binary_logloss: 0.413695
Early stopping, best iteration is:
[1935]	valid_0's binary_logloss: 0.403403	valid_1's binary_logloss: 0.413686
[0.41368577597892464]
cv: 0.41368577597892464
Training un

In [42]:
first=3500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

3370
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.416198	valid_1's binary_logloss: 0.415972
[400]	valid_0's binary_logloss: 0.413916	valid_1's binary_logloss: 0.414822


KeyboardInterrupt: 

In [ ]:
first=4500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=1000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=5000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [44]:
first=2000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))

In [45]:
train_csr_save=train_csr[:,retain_index]
predict_csr_save=predict_csr[:,retain_index]

In [46]:
save_csr(train_csr_save,'../../data/features/xgb/train_csr_2000')
save_csr(predict_csr_save,'../../data/features/xgb/predict_csr_2000')

data save
indices save
indptr save
save done!
data save
indices save
indptr save
save done!
